In [415]:
#Read input graph and data
def read_initial_node(): 
    initial_node = int(input('Enter Start node {0,1,2..}: '))
    return initial_node

def read_goal_node():
    goal_node = int(input('Enter goal node {0,1,2..}: '))
    return goal_node
    
def read_number_of_nodes():
    global nodes_count
    nodes_count = int(input('Enter number of nodes : '))
    return nodes_count

def read_graph():
    global transition_matrix
    transition_matrix = np.matrix(np.ones((nodes_count,nodes_count))*np.inf)
    for each_node in range(nodes_count):
        transition_matrix[each_node,each_node]=0
        edges_with_cost = input('Enter edges from node {0} : '.format(each_node)).split(',')
        for each_edge in edges_with_cost:
            if each_edge:
                transition_matrix[each_node,int(each_edge.split(':')[0])]=int(each_edge.split(':')[1])
                transition_matrix[int(each_edge.split(':')[0]),each_node]=int(each_edge.split(':')[1])
    return transition_matrix

In [408]:
#checking goal node and getting adjacent nodes
def node_reach_goal_node(path,goal_node):
    if path[-1] == goal_node:
        return True
    return False

def get_adj_nodes(nodes):
    adj_nodes = []
    for adj_node in range(nodes_count):
        if transition_matrix[nodes[-1],adj_node] not in (0,float('inf')) and adj_node not in nodes:
            adj_nodes.append(nodes + [adj_node])
    return adj_nodes

In [409]:
#finding cost and displaying path
def find_cost(path):
    cost=0
    for index in range(len(path)-1):
        cost += transition_matrix[path[index],path[index+1]]
    return cost
def display_path(path):
    print('path : ',end='')
    print('->'.join(chr(node+65) for node in path))


In [410]:
#DEPTH FIRST SEARCH
def dfs_algorithm(dfs_stack,goal_node):
    if node_reach_goal_node(dfs_stack[-1],goal_node):
        return dfs_stack[-1]
    else:
        nodes = dfs_stack.pop()
        adj_nodes = get_adj_nodes(nodes)
        if len(adj_nodes) > 0:
            dfs_stack += adj_nodes
        return dfs_algorithm(dfs_stack,goal_node)
    
def dfs(initial_node,goal_node):
    path = dfs_algorithm([[initial_node]],goal_node)
    display_path(path)
    print('cost : ',find_cost(path))

In [411]:
#Hueristic Design
def all_possible_paths(dfs_stack,goal_node,paths=[]):
    if len(dfs_stack) == 0:
        total_cost = 0
        for nodes in paths:
            total_cost += find_cost(nodes)
        if len(paths) == 0:
            return 0;
        avg_cost = total_cost/len(paths)
        return round(avg_cost,2)
    elif node_reach_goal_node(dfs_stack[-1],goal_node):
        #display_path(dfs_stack[-1])
        paths.append(dfs_stack[-1])
        dfs_stack.pop()
        return all_possible_paths(dfs_stack,goal_node,paths)
    else:
        nodes = dfs_stack.pop()
        adj_nodes = get_adj_nodes(nodes)
        if len(adj_nodes) > 0:
            dfs_stack += adj_nodes
        return all_possible_paths(dfs_stack,goal_node,paths)

def find_hueristic_values(goal_node):
    hue_values = []
    for node in range(nodes_count):
        avg_cost = all_possible_paths([[node]],goal_node,[])
        hue_values.append(avg_cost)
    return hue_values

def sort_stack_by_cost(dfs_stack,hue_values):
    for i in range(len(dfs_stack)):
        for j in range(i+1,len(dfs_stack)):
            if (find_cost(dfs_stack[i])+hue_values[dfs_stack[i][-1]])<(find_cost(dfs_stack[j])+hue_values[dfs_stack[j][-1]]):
                temp=dfs_stack[i]
                dfs_stack[i]=dfs_stack[j]
                dfs_stack[j]=temp
    return dfs_stack

In [412]:
#A_STAR
def a_star_algorithm(dfs_stack,hue_values,goal_node):
    if node_reach_goal_node(dfs_stack[-1],goal_node):
        return dfs_stack[-1]
    else:
        nodes = dfs_stack.pop()
        adj_nodes = get_adj_nodes(nodes)
        if len(adj_nodes) > 0:
            dfs_stack += adj_nodes
        dfs_stack = sort_stack_by_cost(dfs_stack,hue_values)
        return a_star_algorithm(dfs_stack,hue_values,goal_node)

def a_star(initial_node,goal_node):
    hue_values = find_hueristic_values(goal_node)
    print("hueristic values : ",hue_values)
    path = a_star_algorithm([[initial_node]],hue_values,goal_node)
    display_path(path)
    print('cost : ',find_cost(path))

In [413]:
#Read input graph
def read_data():
    read_number_of_nodes()
    read_graph()

#Main method
def main():
    #read_data()
    initial_node = read_initial_node()
    goal_node = read_goal_node()
    print("---------DFS-----------")
    dfs(initial_node,goal_node)
    print("---------A star-----------")
    a_star(initial_node,goal_node)

In [416]:
#Start from here
if __name__ == '__main__':
    main()

Enter Start node {0,1,2..}: 0
Enter goal node {0,1,2..}: 3
---------DFS-----------
path : A->G->H->F->D
cost :  51.0
---------A star-----------
hueristic values :  [39.67, 49.67, 40.75, 0.0, 2.0, 50.17, 34.5, 42.83]
path : A->D
cost :  17.0
